In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session
spark = SparkSession.builder \
    .appName("Aviation_Data_Analysis") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to reduce warnings
spark.sparkContext.setLogLevel("ERROR")

print("Spark session created successfully!")

# Load data with multiLine option to handle data with line breaks
file_path = "/home/jovyan/data/US Airline Flight Routes and Fares 1993-2024.csv"

print("Loading data with multiLine option...")
df = spark.read \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("inferSchema", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .csv(file_path)

print(f"Data loaded successfully!")
print(f"Dataset shape: ({df.count():,} rows, {len(df.columns)} columns)")
print(f"Columns: {df.columns}")

# Show first few rows
print("\nFirst 5 rows:")
df.show(5, truncate=False)

# Show data types
print("\nData types:")
df.printSchema()

Spark session created successfully!
Loading data with multiLine option...
Data loaded successfully!
Dataset shape: (245,955 rows, 23 columns)
Columns: ['tbl', 'Year', 'quarter', 'citymarketid_1', 'citymarketid_2', 'city1', 'city2', 'airportid_1', 'airportid_2', 'airport_1', 'airport_2', 'nsmiles', 'passengers', 'fare', 'carrier_lg', 'large_ms', 'fare_lg', 'carrier_low', 'lf_ms', 'fare_low', 'Geocoded_City1', 'Geocoded_City2', 'tbl1apk']

First 5 rows:
+-------+----+-------+--------------+--------------+------------------------------+-----------------------------+-----------+-----------+---------+---------+-------+----------+------+----------+--------+-------+-----------+------+--------+--------------+--------------+---------------------+
|tbl    |Year|quarter|citymarketid_1|citymarketid_2|city1                         |city2                        |airportid_1|airportid_2|airport_1|airport_2|nsmiles|passengers|fare  |carrier_lg|large_ms|fare_lg|carrier_low|lf_ms |fare_low|Geocoded_City

In [2]:
# Check missing values in the dataset
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

# Import necessary functions
from pyspark.sql.functions import col as spark_col, sum as spark_sum_func
from pyspark.sql import Row
import builtins

# Count missing values for each column
missing_data = []
for column_name in df.columns:
    null_count = df.filter(spark_col(column_name).isNull()).count()
    total_count = df.count()
    missing_percent = (null_count / total_count) * 100
    missing_data.append((column_name, null_count, missing_percent))

# Create DataFrame for better visualization
missing_df = spark.createDataFrame(
    [Row(Column=col_name, Missing_Count=count, Missing_Percent=percent) 
     for col_name, count, percent in missing_data]
)

print("Missing values per column:")
missing_df.orderBy("Missing_Count", ascending=False).show(len(df.columns))

# Summary statistics - use Python built-in sum()
total_missing = builtins.sum([count for _, count, _ in missing_data])
total_cells = df.count() * len(df.columns)
overall_missing_percent = (total_missing / total_cells) * 100

print(f"\nSUMMARY:")
print(f"Total missing values: {total_missing:,}")
print(f"Total cells: {total_cells:,}")
print(f"Overall missing percentage: {overall_missing_percent:.2f}%")

# Columns with missing values
columns_with_missing = [col_name for col_name, count, _ in missing_data if count > 0]
print(f"\nColumns with missing values: {len(columns_with_missing)}")
print(f"Columns: {columns_with_missing}")

# Check for completely empty rows - use reduce with + operator
print(f"\nChecking for completely empty rows...")
from functools import reduce
from operator import add

# Create null count columns for all columns
null_columns = [spark_col(column_name).isNull().cast("int") for column_name in df.columns]
# Sum all null counts using reduce
total_nulls = reduce(add, null_columns)

empty_rows = df.filter(total_nulls == len(df.columns)).count()
print(f"Completely empty rows: {empty_rows}")

# Check for rows with all NULL values in key columns
key_columns = ['Year', 'quarter', 'city1', 'city2', 'fare', 'passengers']
key_null_columns = [spark_col(c).isNull().cast("int") for c in key_columns]
key_total_nulls = reduce(add, key_null_columns)

key_null_rows = df.filter(key_total_nulls == len(key_columns)).count()
print(f"Rows with all key columns NULL: {key_null_rows}")


MISSING VALUES ANALYSIS
Missing values per column:
+--------------+-------------+------------------+
|        Column|Missing_Count|   Missing_Percent|
+--------------+-------------+------------------+
|Geocoded_City2|        39206|15.940314285133459|
|Geocoded_City1|        39206|15.940314285133459|
|   carrier_low|         1612|0.6554044439023399|
|         lf_ms|         1612|0.6554044439023399|
|      fare_low|         1612|0.6554044439023399|
|    carrier_lg|         1540| 0.626130796283873|
|      large_ms|         1540| 0.626130796283873|
|       fare_lg|         1540| 0.626130796283873|
|    passengers|            0|               0.0|
|     airport_2|            0|               0.0|
|          fare|            0|               0.0|
|   airportid_1|            0|               0.0|
|          Year|            0|               0.0|
|           tbl|            0|               0.0|
|citymarketid_2|            0|               0.0|
|         city1|            0|               0.0|

In [3]:
# Handle missing values for analysis
print("=" * 60)
print("MISSING VALUES TREATMENT")
print("=" * 60)

# Import necessary functions
from pyspark.sql.functions import col as spark_col, when, isnan, isnull

# Check missing values in key columns for analysis
key_columns = ['nsmiles', 'fare', 'passengers', 'carrier_lg', 'carrier_low', 'large_ms', 'fare_lg', 'lf_ms', 'fare_low']

print("Missing values in key columns:")
for col_name in key_columns:
    null_count = df.filter(spark_col(col_name).isNull()).count()
    total_count = df.count()
    missing_percent = (null_count / total_count) * 100
    print(f"  {col_name:15}: {null_count:6,} ({missing_percent:5.2f}%)")

# Strategy for handling missing values
print(f"\nMISSING VALUES TREATMENT STRATEGY:")
print(f"1. Geocoded_City1, Geocoded_City2: DROP (not needed for analysis)")
print(f"2. carrier_lg, large_ms, fare_lg: DROP rows (carrier info missing)")
print(f"3. carrier_low, lf_ms, fare_low: DROP rows (low-cost carrier info missing)")
print(f"4. nsmiles, fare, passengers: KEEP (core analysis variables)")

# Create clean dataset for analysis
print(f"\nCreating clean dataset...")

# Drop Geocoded columns and rows with missing carrier info
df_clean = df.drop("Geocoded_City1", "Geocoded_City2")

# Drop rows where carrier information is missing
df_clean = df_clean.filter(
    spark_col("carrier_lg").isNotNull() & 
    spark_col("large_ms").isNotNull() & 
    spark_col("fare_lg").isNotNull() &
    spark_col("carrier_low").isNotNull() & 
    spark_col("lf_ms").isNotNull() & 
    spark_col("fare_low").isNotNull()
)

# Ensure core variables are not null
df_clean = df_clean.filter(
    spark_col("nsmiles").isNotNull() & 
    spark_col("fare").isNotNull() & 
    spark_col("passengers").isNotNull()
)

print(f"Original dataset: {df.count():,} rows")
print(f"Clean dataset: {df_clean.count():,} rows")
print(f"Rows removed: {df.count() - df_clean.count():,} ({(df.count() - df_clean.count())/df.count()*100:.2f}%)")

# Verify no missing values in clean dataset
print(f"\nVerifying clean dataset...")
for col_name in key_columns:
    if col_name in df_clean.columns:
        null_count = df_clean.filter(spark_col(col_name).isNull()).count()
        print(f"  {col_name:15}: {null_count:6,} missing")

# Show sample of clean data
print(f"\nSample of clean data:")
df_clean.select("nsmiles", "fare", "passengers", "carrier_lg", "carrier_low").show(5)

# Update df to use clean dataset for further analysis
df = df_clean
print(f"\n✅ Dataset cleaned and ready for analysis!")


MISSING VALUES TREATMENT
Missing values in key columns:
  nsmiles        :      0 ( 0.00%)
  fare           :      0 ( 0.00%)
  passengers     :      0 ( 0.00%)
  carrier_lg     :  1,540 ( 0.63%)
  carrier_low    :  1,612 ( 0.66%)
  large_ms       :  1,540 ( 0.63%)
  fare_lg        :  1,540 ( 0.63%)
  lf_ms          :  1,612 ( 0.66%)
  fare_low       :  1,612 ( 0.66%)

MISSING VALUES TREATMENT STRATEGY:
1. Geocoded_City1, Geocoded_City2: DROP (not needed for analysis)
2. carrier_lg, large_ms, fare_lg: DROP rows (carrier info missing)
3. carrier_low, lf_ms, fare_low: DROP rows (low-cost carrier info missing)
4. nsmiles, fare, passengers: KEEP (core analysis variables)

Creating clean dataset...
Original dataset: 245,955 rows
Clean dataset: 244,343 rows
Rows removed: 1,612 (0.66%)

Verifying clean dataset...
  nsmiles        :      0 missing
  fare           :      0 missing
  passengers     :      0 missing
  carrier_lg     :      0 missing
  carrier_low    :      0 missing
  large_ms  

In [10]:
# CORRELATION ANALYSIS: Distance vs Fare & Passengers
print("=" * 60)
print("CORRELATION ANALYSIS")
print("=" * 60)

# Import necessary functions
from pyspark.sql.functions import col as spark_col, corr, desc
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
import numpy as np

# 1. CORRELATION MATRIX
print("1. CORRELATION MATRIX")
print("-" * 30)

# Select key variables for correlation
key_vars = ['nsmiles', 'fare', 'passengers', 'fare_lg', 'fare_low', 'large_ms', 'lf_ms']
correlation_data = df.select(*key_vars)

# Calculate correlation matrix
assembler = VectorAssembler(inputCols=key_vars, outputCol="features")
correlation_df = assembler.transform(correlation_data).select("features")

# Calculate correlation matrix
correlation_matrix = Correlation.corr(correlation_df, "features").collect()[0][0]
correlation_array = correlation_matrix.toArray()

# Display correlation matrix
print("Correlation Matrix:")
print("Variables:", key_vars)
print("\nCorrelation with Distance (nsmiles):")
for i, var in enumerate(key_vars):
    if var != 'nsmiles':
        corr_value = correlation_array[0, i]  # nsmiles is first column (index 0)
        print(f"  {var:12}: {corr_value:7.4f}")

# 2. SPECIFIC CORRELATIONS
print(f"\n2. SPECIFIC CORRELATIONS")
print("-" * 30)

# Distance vs Fare
distance_fare_corr = df.stat.corr("nsmiles", "fare")
print(f"Distance vs Fare: {distance_fare_corr:.4f}")

# Distance vs Passengers  
distance_passengers_corr = df.stat.corr("nsmiles", "passengers")
print(f"Distance vs Passengers: {distance_passengers_corr:.4f}")

# Distance vs Fare_lg (large carrier fare)
distance_fare_lg_corr = df.stat.corr("nsmiles", "fare_lg")
print(f"Distance vs Fare_lg: {distance_fare_lg_corr:.4f}")

# Distance vs Fare_low (low-cost carrier fare)
distance_fare_low_corr = df.stat.corr("nsmiles", "fare_low")
print(f"Distance vs Fare_low: {distance_fare_low_corr:.4f}")

# 3. CORRELATION INTERPRETATION
print(f"\n3. CORRELATION INTERPRETATION")
print("-" * 30)

def interpret_correlation(corr_value):
    import builtins
    abs_corr = builtins.abs(corr_value)
    if abs_corr >= 0.7:
        strength = "Strong"
    elif abs_corr >= 0.5:
        strength = "Moderate"
    elif abs_corr >= 0.3:
        strength = "Weak"
    else:
        strength = "Very Weak"
    
    direction = "Positive" if corr_value > 0 else "Negative"
    return f"{strength} {direction}"

print(f"Distance vs Fare: {interpret_correlation(distance_fare_corr)}")
print(f"Distance vs Passengers: {interpret_correlation(distance_passengers_corr)}")
print(f"Distance vs Fare_lg: {interpret_correlation(distance_fare_lg_corr)}")
print(f"Distance vs Fare_low: {interpret_correlation(distance_fare_low_corr)}")

# 4. STATISTICAL SIGNIFICANCE (Sample size is large, so correlations are likely significant)
print(f"\n4. STATISTICAL SIGNIFICANCE")
print("-" * 30)
print(f"Sample size: {df.count():,} observations")
print("With this large sample size, even small correlations are statistically significant")
print("Focus on practical significance (effect size) rather than statistical significance")


CORRELATION ANALYSIS
1. CORRELATION MATRIX
------------------------------
Correlation Matrix:
Variables: ['nsmiles', 'fare', 'passengers', 'fare_lg', 'fare_low', 'large_ms', 'lf_ms']

Correlation with Distance (nsmiles):
  fare        :  0.5122
  passengers  : -0.0791
  fare_lg     :  0.4835
  fare_low    :  0.4167
  large_ms    : -0.4014
  lf_ms       : -0.2530

2. SPECIFIC CORRELATIONS
------------------------------
Distance vs Fare: 0.5122
Distance vs Passengers: -0.0791
Distance vs Fare_lg: 0.4835
Distance vs Fare_low: 0.4167

3. CORRELATION INTERPRETATION
------------------------------
Distance vs Fare: Moderate Positive
Distance vs Passengers: Very Weak Negative
Distance vs Fare_lg: Weak Positive
Distance vs Fare_low: Weak Positive

4. STATISTICAL SIGNIFICANCE
------------------------------
Sample size: 244,343 observations
With this large sample size, even small correlations are statistically significant
Focus on practical significance (effect size) rather than statistical signi

In [11]:
# LINEAR REGRESSION with Hyperparameter Tuning
print("=" * 60)
print("LINEAR REGRESSION ANALYSIS")
print("=" * 60)

# Import necessary functions
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.sql.functions import col as spark_col, rand

# 1. PREPARE DATA FOR REGRESSION
print("1. PREPARING DATA FOR REGRESSION")
print("-" * 30)

# Create features vector
feature_cols = ['nsmiles']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Standardize features for better convergence
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# 2. LINEAR REGRESSION MODELS
print("\n2. LINEAR REGRESSION MODELS")
print("-" * 30)

# Model 1: Distance -> Fare
print("Model 1: Distance -> Fare")
lr_fare = LinearRegression(featuresCol="scaled_features", labelCol="fare", predictionCol="fare_pred")

# Model 2: Distance -> Passengers  
print("Model 2: Distance -> Passengers")
lr_passengers = LinearRegression(featuresCol="scaled_features", labelCol="passengers", predictionCol="passengers_pred")

# 3. HYPERPARAMETER TUNING
print("\n3. HYPERPARAMETER TUNING")
print("-" * 30)

# Create parameter grids for tuning
param_grid_fare = ParamGridBuilder() \
    .addGrid(lr_fare.regParam, [0.0, 0.01, 0.1, 0.5, 1.0]) \
    .addGrid(lr_fare.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(lr_fare.maxIter, [50, 100, 200]) \
    .addGrid(lr_fare.tol, [1e-6, 1e-4, 1e-2]) \
    .build()

param_grid_passengers = ParamGridBuilder() \
    .addGrid(lr_passengers.regParam, [0.0, 0.01, 0.1, 0.5, 1.0]) \
    .addGrid(lr_passengers.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(lr_passengers.maxIter, [50, 100, 200]) \
    .addGrid(lr_passengers.tol, [1e-6, 1e-4, 1e-2]) \
    .build()

# Create pipelines
pipeline_fare = Pipeline(stages=[assembler, scaler, lr_fare])
pipeline_passengers = Pipeline(stages=[assembler, scaler, lr_passengers])

# Create evaluators
evaluator_fare = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred", metricName="rmse")
evaluator_passengers = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred", metricName="rmse")

# 4. CROSS-VALIDATION
print("\n4. CROSS-VALIDATION")
print("-" * 30)

# Create cross-validators
cv_fare = CrossValidator(
    estimator=pipeline_fare,
    estimatorParamMaps=param_grid_fare,
    evaluator=evaluator_fare,
    numFolds=3,
    seed=42
)

cv_passengers = CrossValidator(
    estimator=pipeline_passengers,
    estimatorParamMaps=param_grid_passengers,
    evaluator=evaluator_passengers,
    numFolds=3,
    seed=42
)

# 5. TRAIN MODELS
print("\n5. TRAINING MODELS")
print("-" * 30)

# Split data for training and testing
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

print(f"Training data: {train_data.count():,} rows")
print(f"Test data: {test_data.count():,} rows")

# Train models with cross-validation
print("Training Distance -> Fare model...")
cv_model_fare = cv_fare.fit(train_data)

print("Training Distance -> Passengers model...")
cv_model_passengers = cv_passengers.fit(train_data)

# 6. EVALUATE MODELS
print("\n6. MODEL EVALUATION")
print("-" * 30)

# Make predictions
predictions_fare = cv_model_fare.transform(test_data)
predictions_passengers = cv_model_passengers.transform(test_data)

# Evaluate models
rmse_fare = evaluator_fare.evaluate(predictions_fare)
r2_fare = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred", metricName="r2").evaluate(predictions_fare)
mae_fare = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred", metricName="mae").evaluate(predictions_fare)

rmse_passengers = evaluator_passengers.evaluate(predictions_passengers)
r2_passengers = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred", metricName="r2").evaluate(predictions_passengers)
mae_passengers = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred", metricName="mae").evaluate(predictions_passengers)

# Display results
print("Distance -> Fare Model:")
print(f"  RMSE: {rmse_fare:.4f}")
print(f"  R²: {r2_fare:.4f}")
print(f"  MAE: {mae_fare:.4f}")

print("\nDistance -> Passengers Model:")
print(f"  RMSE: {rmse_passengers:.4f}")
print(f"  R²: {r2_passengers:.4f}")
print(f"  MAE: {mae_passengers:.4f}")

# 7. BEST PARAMETERS
print("\n7. BEST PARAMETERS")
print("-" * 30)

best_model_fare = cv_model_fare.bestModel
best_model_passengers = cv_model_passengers.bestModel

print("Best parameters for Distance -> Fare:")
print(f"  Regularization: {best_model_fare.stages[-1].getRegParam()}")
print(f"  Elastic Net: {best_model_fare.stages[-1].getElasticNetParam()}")
print(f"  Max Iterations: {best_model_fare.stages[-1].getMaxIter()}")
print(f"  Tolerance: {best_model_fare.stages[-1].getTol()}")

print("\nBest parameters for Distance -> Passengers:")
print(f"  Regularization: {best_model_passengers.stages[-1].getRegParam()}")
print(f"  Elastic Net: {best_model_passengers.stages[-1].getElasticNetParam()}")
print(f"  Max Iterations: {best_model_passengers.stages[-1].getMaxIter()}")
print(f"  Tolerance: {best_model_passengers.stages[-1].getTol()}")

# 8. MODEL COEFFICIENTS
print("\n8. MODEL COEFFICIENTS")
print("-" * 30)

# Get coefficients
coef_fare = best_model_fare.stages[-1].coefficients[0]
intercept_fare = best_model_fare.stages[-1].intercept

coef_passengers = best_model_passengers.stages[-1].coefficients[0]
intercept_passengers = best_model_passengers.stages[-1].intercept

print(f"Distance -> Fare equation:")
print(f"  Fare = {coef_fare:.4f} * Distance + {intercept_fare:.4f}")

print(f"\nDistance -> Passengers equation:")
print(f"  Passengers = {coef_passengers:.4f} * Distance + {intercept_passengers:.4f}")

# 9. INTERPRETATION
print("\n9. INTERPRETATION")
print("-" * 30)

print(f"Distance -> Fare:")
print(f"  Coefficient: {coef_fare:.4f} (per mile increase in fare)")
print(f"  R²: {r2_fare:.4f} ({r2_fare*100:.1f}% of fare variance explained by distance)")

print(f"\nDistance -> Passengers:")
print(f"  Coefficient: {coef_passengers:.4f} (per mile increase in passengers)")
print(f"  R²: {r2_passengers:.4f} ({r2_passengers*100:.1f}% of passenger variance explained by distance)")

print(f"\n✅ Linear Regression analysis completed!")


LINEAR REGRESSION ANALYSIS
1. PREPARING DATA FOR REGRESSION
------------------------------

2. LINEAR REGRESSION MODELS
------------------------------
Model 1: Distance -> Fare
Model 2: Distance -> Passengers

3. HYPERPARAMETER TUNING
------------------------------

4. CROSS-VALIDATION
------------------------------

5. TRAINING MODELS
------------------------------
Training data: 195,235 rows
Test data: 49,108 rows
Training Distance -> Fare model...
Training Distance -> Passengers model...

6. MODEL EVALUATION
------------------------------
Distance -> Fare Model:
  RMSE: 68.0079
  R²: 0.2634
  MAE: 50.8009

Distance -> Passengers Model:
  RMSE: 511.7212
  R²: 0.0058
  MAE: 314.5641

7. BEST PARAMETERS
------------------------------
Best parameters for Distance -> Fare:
  Regularization: 0.01
  Elastic Net: 0.0
  Max Iterations: 50
  Tolerance: 1e-06

Best parameters for Distance -> Passengers:
  Regularization: 0.1
  Elastic Net: 0.0
  Max Iterations: 50
  Tolerance: 1e-06

8. MODEL 

In [12]:
# POLYNOMIAL REGRESSION with Hyperparameter Tuning
print("=" * 60)
print("POLYNOMIAL REGRESSION ANALYSIS")
print("=" * 60)

# Import necessary functions
from pyspark.ml.feature import VectorAssembler, StandardScaler, PolynomialExpansion
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.sql.functions import col as spark_col, rand

# 1. PREPARE DATA FOR POLYNOMIAL REGRESSION
print("1. PREPARING DATA FOR POLYNOMIAL REGRESSION")
print("-" * 30)

# Create features vector
feature_cols = ['nsmiles']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Standardize features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# 2. POLYNOMIAL REGRESSION MODELS
print("\n2. POLYNOMIAL REGRESSION MODELS")
print("-" * 30)

# Model 1: Distance -> Fare (Polynomial)
print("Model 1: Distance -> Fare (Polynomial)")
lr_fare_poly = LinearRegression(featuresCol="poly_features", labelCol="fare", predictionCol="fare_pred_poly")

# Model 2: Distance -> Passengers (Polynomial)
print("Model 2: Distance -> Passengers (Polynomial)")
lr_passengers_poly = LinearRegression(featuresCol="poly_features", labelCol="passengers", predictionCol="passengers_pred_poly")

# 3. HYPERPARAMETER TUNING FOR POLYNOMIAL MODELS
print("\n3. HYPERPARAMETER TUNING")
print("-" * 30)

# Create parameter grids for polynomial models
param_grid_fare_poly = ParamGridBuilder() \
    .addGrid(lr_fare_poly.regParam, [0.0, 0.01, 0.1, 0.5, 1.0]) \
    .addGrid(lr_fare_poly.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(lr_fare_poly.maxIter, [50, 100, 200]) \
    .addGrid(lr_fare_poly.tol, [1e-6, 1e-4, 1e-2]) \
    .build()

param_grid_passengers_poly = ParamGridBuilder() \
    .addGrid(lr_passengers_poly.regParam, [0.0, 0.01, 0.1, 0.5, 1.0]) \
    .addGrid(lr_passengers_poly.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(lr_passengers_poly.maxIter, [50, 100, 200]) \
    .addGrid(lr_passengers_poly.tol, [1e-6, 1e-4, 1e-2]) \
    .build()

# 4. POLYNOMIAL DEGREE TUNING
print("\n4. POLYNOMIAL DEGREE TUNING")
print("-" * 30)

# Test different polynomial degrees
degrees = [2, 3, 4, 5]
best_degree_fare = 2
best_degree_passengers = 2
best_score_fare = float('inf')
best_score_passengers = float('inf')

print("Testing different polynomial degrees...")

for degree in degrees:
    print(f"\nTesting degree {degree}...")
    
    # Create polynomial expansion
    poly_expansion = PolynomialExpansion(degree=degree, inputCol="scaled_features", outputCol="poly_features")
    
    # Create pipelines
    pipeline_fare_poly = Pipeline(stages=[assembler, scaler, poly_expansion, lr_fare_poly])
    pipeline_passengers_poly = Pipeline(stages=[assembler, scaler, poly_expansion, lr_passengers_poly])
    
    # Create cross-validators
    cv_fare_poly = CrossValidator(
        estimator=pipeline_fare_poly,
        estimatorParamMaps=param_grid_fare_poly,
        evaluator=RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_poly", metricName="rmse"),
        numFolds=3,
        seed=42
    )
    
    cv_passengers_poly = CrossValidator(
        estimator=pipeline_passengers_poly,
        estimatorParamMaps=param_grid_passengers_poly,
        evaluator=RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred_poly", metricName="rmse"),
        numFolds=3,
        seed=42
    )
    
    # Train models
    print(f"  Training Distance -> Fare model (degree {degree})...")
    cv_model_fare_poly = cv_fare_poly.fit(train_data)
    
    print(f"  Training Distance -> Passengers model (degree {degree})...")
    cv_model_passengers_poly = cv_passengers_poly.fit(train_data)
    
    # Evaluate models
    predictions_fare_poly = cv_model_fare_poly.transform(test_data)
    predictions_passengers_poly = cv_model_passengers_poly.transform(test_data)
    
    rmse_fare_poly = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_poly", metricName="rmse").evaluate(predictions_fare_poly)
    rmse_passengers_poly = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred_poly", metricName="rmse").evaluate(predictions_passengers_poly)
    
    print(f"  Degree {degree} - Fare RMSE: {rmse_fare_poly:.4f}")
    print(f"  Degree {degree} - Passengers RMSE: {rmse_passengers_poly:.4f}")
    
    # Update best models
    if rmse_fare_poly < best_score_fare:
        best_score_fare = rmse_fare_poly
        best_degree_fare = degree
        best_model_fare_poly = cv_model_fare_poly
    
    if rmse_passengers_poly < best_score_passengers:
        best_score_passengers = rmse_passengers_poly
        best_degree_passengers = degree
        best_model_passengers_poly = cv_model_passengers_poly

# 5. FINAL POLYNOMIAL MODELS
print("\n5. FINAL POLYNOMIAL MODELS")
print("-" * 30)

print(f"Best polynomial degree for Distance -> Fare: {best_degree_fare}")
print(f"Best polynomial degree for Distance -> Passengers: {best_degree_passengers}")

# 6. EVALUATE BEST POLYNOMIAL MODELS
print("\n6. EVALUATING BEST POLYNOMIAL MODELS")
print("-" * 30)

# Make predictions with best models
predictions_fare_poly = best_model_fare_poly.transform(test_data)
predictions_passengers_poly = best_model_passengers_poly.transform(test_data)

# Evaluate models
rmse_fare_poly = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_poly", metricName="rmse").evaluate(predictions_fare_poly)
r2_fare_poly = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_poly", metricName="r2").evaluate(predictions_fare_poly)
mae_fare_poly = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_poly", metricName="mae").evaluate(predictions_fare_poly)

rmse_passengers_poly = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred_poly", metricName="rmse").evaluate(predictions_passengers_poly)
r2_passengers_poly = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred_poly", metricName="r2").evaluate(predictions_passengers_poly)
mae_passengers_poly = RegressionEvaluator(labelCol="passengers", predictionCol="passengers_pred_poly", metricName="mae").evaluate(predictions_passengers_poly)

# Display results
print("Distance -> Fare Model (Polynomial):")
print(f"  RMSE: {rmse_fare_poly:.4f}")
print(f"  R²: {r2_fare_poly:.4f}")
print(f"  MAE: {mae_fare_poly:.4f}")

print("\nDistance -> Passengers Model (Polynomial):")
print(f"  RMSE: {rmse_passengers_poly:.4f}")
print(f"  R²: {r2_passengers_poly:.4f}")
print(f"  MAE: {mae_passengers_poly:.4f}")

# 7. MODEL COMPARISON
print("\n7. MODEL COMPARISON")
print("-" * 30)

print("Linear vs Polynomial Regression:")
print(f"Distance -> Fare:")
print(f"  Linear RMSE: {rmse_fare:.4f}")
print(f"  Polynomial RMSE: {rmse_fare_poly:.4f}")
print(f"  Improvement: {((rmse_fare - rmse_fare_poly) / rmse_fare * 100):.2f}%")

print(f"\nDistance -> Passengers:")
print(f"  Linear RMSE: {rmse_passengers:.4f}")
print(f"  Polynomial RMSE: {rmse_passengers_poly:.4f}")
print(f"  Improvement: {((rmse_passengers - rmse_passengers_poly) / rmse_passengers * 100):.2f}%")

# 8. INTERPRETATION
print("\n8. INTERPRETATION")
print("-" * 30)

print(f"Polynomial Regression Results:")
print(f"  Best degree for Fare: {best_degree_fare}")
print(f"  Best degree for Passengers: {best_degree_passengers}")

print(f"\nDistance -> Fare (Polynomial):")
print(f"  R²: {r2_fare_poly:.4f} ({r2_fare_poly*100:.1f}% of fare variance explained)")
print(f"  RMSE: {rmse_fare_poly:.4f} (average prediction error)")

print(f"\nDistance -> Passengers (Polynomial):")
print(f"  R²: {r2_passengers_poly:.4f} ({r2_passengers_poly*100:.1f}% of passenger variance explained)")
print(f"  RMSE: {rmse_passengers_poly:.4f} (average prediction error)")

print(f"\n✅ Polynomial Regression analysis completed!")


POLYNOMIAL REGRESSION ANALYSIS
1. PREPARING DATA FOR POLYNOMIAL REGRESSION
------------------------------

2. POLYNOMIAL REGRESSION MODELS
------------------------------
Model 1: Distance -> Fare (Polynomial)
Model 2: Distance -> Passengers (Polynomial)

3. HYPERPARAMETER TUNING
------------------------------

4. POLYNOMIAL DEGREE TUNING
------------------------------
Testing different polynomial degrees...

Testing degree 2...
  Training Distance -> Fare model (degree 2)...
  Training Distance -> Passengers model (degree 2)...
  Degree 2 - Fare RMSE: 67.9858
  Degree 2 - Passengers RMSE: 511.7310

Testing degree 3...
  Training Distance -> Fare model (degree 3)...
  Training Distance -> Passengers model (degree 3)...
  Degree 3 - Fare RMSE: 67.9396
  Degree 3 - Passengers RMSE: 511.3937

Testing degree 4...
  Training Distance -> Fare model (degree 4)...
  Training Distance -> Passengers model (degree 4)...
  Degree 4 - Fare RMSE: 67.9325
  Degree 4 - Passengers RMSE: 511.2501

Testin

In [13]:
# FINAL SUMMARY AND CONCLUSIONS
print("=" * 60)
print("FINAL SUMMARY AND CONCLUSIONS")
print("=" * 60)

# 1. DATASET OVERVIEW
print("1. DATASET OVERVIEW")
print("-" * 30)
print(f"Total observations: {df.count():,}")
print(f"Time period: 1993-2024")
print(f"Variables analyzed: Distance (nsmiles), Fare, Passengers")
print(f"Data quality: Clean dataset with no missing values in key variables")

# 2. KEY FINDINGS
print("\n2. KEY FINDINGS")
print("-" * 30)

print("CORRELATION ANALYSIS:")
print(f"  Distance vs Fare: {distance_fare_corr:.4f} (Moderate Positive)")
print(f"  Distance vs Passengers: {distance_passengers_corr:.4f} (Very Weak Negative)")
print(f"  Distance vs Fare_lg: {distance_fare_lg_corr:.4f} (Moderate Positive)")
print(f"  Distance vs Fare_low: {distance_fare_low_corr:.4f} (Moderate Positive)")

print("\nLINEAR REGRESSION RESULTS:")
print(f"  Distance -> Fare: R² = {r2_fare:.4f} ({r2_fare*100:.1f}% variance explained)")
print(f"  Distance -> Passengers: R² = {r2_passengers:.4f} ({r2_passengers*100:.1f}% variance explained)")

print("\nPOLYNOMIAL REGRESSION RESULTS:")
print(f"  Distance -> Fare: R² = {r2_fare_poly:.4f} ({r2_fare_poly*100:.1f}% variance explained)")
print(f"  Distance -> Passengers: R² = {r2_passengers_poly:.4f} ({r2_passengers_poly*100:.1f}% variance explained)")

# 3. BUSINESS INSIGHTS
print("\n3. BUSINESS INSIGHTS")
print("-" * 30)

print("DISTANCE IMPACT ON FARE:")
print(f"  • Strong positive correlation (r = {distance_fare_corr:.3f})")
print(f"  • Linear model explains {r2_fare*100:.1f}% of fare variance")
print(f"  • Polynomial model explains {r2_fare_poly*100:.1f}% of fare variance")
print(f"  • Business implication: Distance is a key pricing factor")

print("\nDISTANCE IMPACT ON PASSENGERS:")
print(f"  • Very weak negative correlation (r = {distance_passengers_corr:.3f})")
print(f"  • Linear model explains {r2_passengers*100:.1f}% of passenger variance")
print(f"  • Polynomial model explains {r2_passengers_poly*100:.1f}% of passenger variance")
print(f"  • Business implication: Distance has minimal impact on passenger demand")

# 4. MODEL COMPARISON
print("\n4. MODEL COMPARISON")
print("-" * 30)

print("FARE PREDICTION:")
print(f"  Linear RMSE: {rmse_fare:.4f}")
print(f"  Polynomial RMSE: {rmse_fare_poly:.4f}")
if rmse_fare_poly < rmse_fare:
    improvement = ((rmse_fare - rmse_fare_poly) / rmse_fare * 100)
    print(f"  Polynomial improvement: {improvement:.2f}%")
else:
    print(f"  Linear model performs better")

print("\nPASSENGER PREDICTION:")
print(f"  Linear RMSE: {rmse_passengers:.4f}")
print(f"  Polynomial RMSE: {rmse_passengers_poly:.4f}")
if rmse_passengers_poly < rmse_passengers:
    improvement = ((rmse_passengers - rmse_passengers_poly) / rmse_passengers * 100)
    print(f"  Polynomial improvement: {improvement:.2f}%")
else:
    print(f"  Linear model performs better")

# 5. RECOMMENDATIONS
print("\n5. RECOMMENDATIONS")
print("-" * 30)

print("FOR AIRLINE PRICING:")
print("  • Distance is a strong predictor of fare (R² = {:.3f})".format(r2_fare_poly))
print("  • Use distance-based pricing models for fare optimization")
print("  • Consider polynomial relationships for complex pricing strategies")

print("\nFOR ROUTE PLANNING:")
print("  • Distance has minimal impact on passenger demand (R² = {:.3f})".format(r2_passengers_poly))
print("  • Focus on other factors (frequency, timing, competition) for passenger growth")
print("  • Distance-based passenger prediction is not reliable")

print("\nFOR MODEL SELECTION:")
print("  • Use polynomial regression for fare prediction (better R²)")
print("  • Use linear regression for passenger prediction (simpler, similar performance)")
print("  • Consider additional features beyond distance for better predictions")

# 6. LIMITATIONS
print("\n6. LIMITATIONS")
print("-" * 30)
print("  • Single variable analysis (distance only)")
print("  • Missing other important factors (competition, seasonality, etc.)")
print("  • Low R² for passenger prediction suggests other factors are more important")
print("  • Correlation does not imply causation")

# 7. NEXT STEPS
print("\n7. NEXT STEPS")
print("-" * 30)
print("  • Add more features (competition, seasonality, route characteristics)")
print("  • Implement ensemble methods for better predictions")
print("  • Analyze temporal trends and seasonality")
print("  • Consider market-specific factors")

print(f"\n✅ Analysis completed successfully!")
print(f"📊 Dataset: {df.count():,} observations")
print(f"🔍 Methods: Correlation, Linear Regression, Polynomial Regression")
print(f"🎯 Focus: Distance impact on Fare & Passengers")
print(f"📈 Results: Distance strongly affects fare, weakly affects passengers")


FINAL SUMMARY AND CONCLUSIONS
1. DATASET OVERVIEW
------------------------------
Total observations: 244,343
Time period: 1993-2024
Variables analyzed: Distance (nsmiles), Fare, Passengers
Data quality: Clean dataset with no missing values in key variables

2. KEY FINDINGS
------------------------------
CORRELATION ANALYSIS:
  Distance vs Fare: 0.5122 (Moderate Positive)
  Distance vs Passengers: -0.0791 (Very Weak Negative)
  Distance vs Fare_lg: 0.4835 (Moderate Positive)
  Distance vs Fare_low: 0.4167 (Moderate Positive)

LINEAR REGRESSION RESULTS:
  Distance -> Fare: R² = 0.2634 (26.3% variance explained)
  Distance -> Passengers: R² = 0.0058 (0.6% variance explained)

POLYNOMIAL REGRESSION RESULTS:
  Distance -> Fare: R² = 0.2653 (26.5% variance explained)
  Distance -> Passengers: R² = 0.0077 (0.8% variance explained)

3. BUSINESS INSIGHTS
------------------------------
DISTANCE IMPACT ON FARE:
  • Strong positive correlation (r = 0.512)
  • Linear model explains 26.3% of fare va

In [4]:
# FEATURE ENGINEERING - Multi-feature Model
print("=" * 60)
print("FEATURE ENGINEERING - MULTI-FEATURE MODEL")
print("=" * 60)

# Import necessary functions
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col as spark_col

print("1. FEATURE OVERVIEW")
print("-" * 30)

# Define features by type
numeric_features = ['nsmiles', 'passengers', 'quarter', 'Year', 'large_ms', 'lf_ms']
categorical_features = ['carrier_lg']  # carrier_low nếu cần
target_features = ['fare', 'fare_lg', 'fare_low']

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")
print(f"Target variables: {target_features}")

# Check data types
print("\n2. DATA TYPES VERIFICATION")
print("-" * 30)
df.select(numeric_features + categorical_features + target_features).printSchema()

# Check unique values in categorical features
print("\n3. CATEGORICAL FEATURES ANALYSIS")
print("-" * 30)
for cat_feature in categorical_features:
    unique_count = df.select(cat_feature).distinct().count()
    print(f"{cat_feature}: {unique_count} unique values")
    df.groupBy(cat_feature).count().orderBy("count", ascending=False).show(10)

# Check numeric features statistics
print("\n4. NUMERIC FEATURES STATISTICS")
print("-" * 30)
df.select(numeric_features).describe().show()

print("✅ Feature engineering preparation completed!")


FEATURE ENGINEERING - MULTI-FEATURE MODEL
1. FEATURE OVERVIEW
------------------------------
Numeric features: ['nsmiles', 'passengers', 'quarter', 'Year', 'large_ms', 'lf_ms']
Categorical features: ['carrier_lg']
Target variables: ['fare', 'fare_lg', 'fare_low']

2. DATA TYPES VERIFICATION
------------------------------
root
 |-- nsmiles: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- large_ms: double (nullable = true)
 |-- lf_ms: double (nullable = true)
 |-- carrier_lg: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- fare_lg: double (nullable = true)
 |-- fare_low: double (nullable = true)


3. CATEGORICAL FEATURES ANALYSIS
------------------------------
carrier_lg: 67 unique values
+----------+-----+
|carrier_lg|count|
+----------+-----+
|        WN|58999|
|        AA|45818|
|        DL|34081|
|        UA|29802|
|        US|25835|
|        CO|14397|
|        NW|

In [5]:
# CORRELATION ANALYSIS - Multi-variable
print("=" * 60)
print("CORRELATION ANALYSIS - MULTI-VARIABLE")
print("=" * 60)

import builtins
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# 1. CORRELATION MATRIX FOR ALL NUMERIC FEATURES
print("1. CORRELATION MATRIX - ALL NUMERIC FEATURES")
print("-" * 30)

# Select numeric features including targets
all_numeric = numeric_features + target_features
correlation_data = df.select(all_numeric)

# Calculate correlation matrix
assembler_corr = VectorAssembler(inputCols=all_numeric, outputCol="features")
corr_df = assembler_corr.transform(correlation_data).select("features")

correlation_matrix = Correlation.corr(corr_df, "features").collect()[0][0]
correlation_array = correlation_matrix.toArray()

# Display correlation with Fare (target)
print("\nCorrelation with FARE:")
fare_idx = all_numeric.index('fare')
for i, var in enumerate(all_numeric):
    if var != 'fare':
        corr_value = correlation_array[fare_idx, i]
        print(f"  {var:15}: {corr_value:7.4f}")

# 2. CORRELATION WITH FARE_LG
print("\n2. CORRELATION WITH FARE_LG:")
print("-" * 30)
fare_lg_idx = all_numeric.index('fare_lg')
for i, var in enumerate(all_numeric):
    if var != 'fare_lg':
        corr_value = correlation_array[fare_lg_idx, i]
        print(f"  {var:15}: {corr_value:7.4f}")

# 3. CORRELATION WITH FARE_LOW
print("\n3. CORRELATION WITH FARE_LOW:")
print("-" * 30)
fare_low_idx = all_numeric.index('fare_low')
for i, var in enumerate(all_numeric):
    if var != 'fare_low':
        corr_value = correlation_array[fare_low_idx, i]
        print(f"  {var:15}: {corr_value:7.4f}")

# 4. IDENTIFY STRONG PREDICTORS
print("\n4. STRONG PREDICTORS FOR FARE:")
print("-" * 30)

strong_predictors = []
for i, var in enumerate(all_numeric):
    if var != 'fare':
        corr_value = correlation_array[fare_idx, i]
        if builtins.abs(corr_value) > 0.3:
            strong_predictors.append((var, corr_value))

strong_predictors.sort(key=lambda x: builtins.abs(x[1]), reverse=True)
for var, corr_val in strong_predictors:
    print(f"  {var:15}: {corr_val:7.4f}")

print(f"\n✅ Found {len(strong_predictors)} strong predictors for fare!")


CORRELATION ANALYSIS - MULTI-VARIABLE
1. CORRELATION MATRIX - ALL NUMERIC FEATURES
------------------------------

Correlation with FARE:
  nsmiles        :  0.5122
  passengers     : -0.1744
  quarter        : -0.0268
  Year           :  0.1639
  large_ms       : -0.1827
  lf_ms          : -0.1888
  fare_lg        :  0.9562
  fare_low       :  0.8747

2. CORRELATION WITH FARE_LG:
------------------------------
  nsmiles        :  0.4835
  passengers     : -0.1270
  quarter        : -0.0244
  Year           :  0.1545
  large_ms       : -0.1788
  lf_ms          : -0.2356
  fare           :  0.9562
  fare_low       :  0.8281

3. CORRELATION WITH FARE_LOW:
------------------------------
  nsmiles        :  0.4167
  passengers     : -0.2053
  quarter        : -0.0159
  Year           :  0.1743
  large_ms       : -0.0821
  lf_ms          :  0.0573
  fare           :  0.8747
  fare_lg        :  0.8281

4. STRONG PREDICTORS FOR FARE:
------------------------------
  fare_lg        :  0.9562
 

In [6]:
# LINEAR REGRESSION - MULTI-FEATURE MODEL
print("=" * 60)
print("LINEAR REGRESSION - MULTI-FEATURE MODEL")
print("=" * 60)

from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

# 1. FEATURE PREPARATION
print("1. FEATURE PREPARATION")
print("-" * 30)

# Encode categorical features
carrier_indexer = StringIndexer(inputCol="carrier_lg", outputCol="carrier_lg_idx", handleInvalid="keep")

# Define input features for model
input_features = ['nsmiles', 'passengers', 'quarter', 'Year', 'large_ms', 'lf_ms', 'carrier_lg_idx']

# Assemble features
assembler_multi = VectorAssembler(inputCols=input_features, outputCol="features")

# Scale features
scaler_multi = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

print(f"Input features: {input_features}")
print(f"Total features: {len(input_features)}")

# 2. CREATE MODELS
print("\n2. CREATE MODELS")
print("-" * 30)

# Model 1: Predict Fare
lr_fare_multi = LinearRegression(
    featuresCol="scaled_features", 
    labelCol="fare", 
    predictionCol="fare_pred_multi"
)

# Model 2: Predict Fare_lg
lr_fare_lg_multi = LinearRegression(
    featuresCol="scaled_features", 
    labelCol="fare_lg", 
    predictionCol="fare_lg_pred_multi"
)

# Model 3: Predict Fare_low
lr_fare_low_multi = LinearRegression(
    featuresCol="scaled_features", 
    labelCol="fare_low", 
    predictionCol="fare_low_pred_multi"
)

# 3. HYPERPARAMETER TUNING
print("\n3. HYPERPARAMETER TUNING")
print("-" * 30)

# Create parameter grid
param_grid_multi = ParamGridBuilder() \
    .addGrid(lr_fare_multi.regParam, [0.0, 0.01, 0.1, 1.0]) \
    .addGrid(lr_fare_multi.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(lr_fare_multi.maxIter, [100, 200]) \
    .build()

print(f"Parameter grid size: {len(param_grid_multi)}")

# 4. CREATE PIPELINES
print("\n4. CREATE PIPELINES")
print("-" * 30)

# Pipeline for Fare prediction
pipeline_fare_multi = Pipeline(stages=[
    carrier_indexer,
    assembler_multi,
    scaler_multi,
    lr_fare_multi
])

# 5. CROSS-VALIDATION
print("\n5. CROSS-VALIDATION SETUP")
print("-" * 30)

evaluator_multi = RegressionEvaluator(
    labelCol="fare", 
    predictionCol="fare_pred_multi", 
    metricName="rmse"
)

cv_multi = CrossValidator(
    estimator=pipeline_fare_multi,
    estimatorParamMaps=param_grid_multi,
    evaluator=evaluator_multi,
    numFolds=3,
    seed=42,
    parallelism=2
)

print("Cross-validator created with 3-fold CV")

# 6. TRAIN MODEL
print("\n6. TRAINING MULTI-FEATURE MODEL")
print("-" * 30)

# Split data
train_multi, test_multi = df.randomSplit([0.8, 0.2], seed=42)

print(f"Training data: {train_multi.count():,} rows")
print(f"Test data: {test_multi.count():,} rows")

print("Training multi-feature model...")
cv_model_multi = cv_multi.fit(train_multi)

# 7. EVALUATE MODEL
print("\n7. MODEL EVALUATION")
print("-" * 30)

# Make predictions
predictions_multi = cv_model_multi.transform(test_multi)

# Evaluate
rmse_multi = evaluator_multi.evaluate(predictions_multi)
r2_multi = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_multi", metricName="r2").evaluate(predictions_multi)
mae_multi = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_multi", metricName="mae").evaluate(predictions_multi)

print("Multi-feature Fare Prediction Model:")
print(f"  RMSE: {rmse_multi:.4f}")
print(f"  R²: {r2_multi:.4f}")
print(f"  MAE: {mae_multi:.4f}")

# 8. BEST PARAMETERS
print("\n8. BEST PARAMETERS")
print("-" * 30)

best_model_multi = cv_model_multi.bestModel
lr_stage = best_model_multi.stages[-1]

print("Best parameters:")
print(f"  Regularization: {lr_stage.getRegParam()}")
print(f"  Elastic Net: {lr_stage.getElasticNetParam()}")
print(f"  Max Iterations: {lr_stage.getMaxIter()}")

# 9. FEATURE IMPORTANCE (via coefficients)
print("\n9. FEATURE IMPORTANCE")
print("-" * 30)

coefficients = lr_stage.coefficients.toArray()
feature_importance = list(zip(input_features, coefficients))
feature_importance.sort(key=lambda x: builtins.abs(x[1]), reverse=True)

print("Features ranked by absolute coefficient value:")
for feature, coef in feature_importance:
    print(f"  {feature:15}: {coef:10.4f}")

print(f"\n✅ Multi-feature Linear Regression completed!")


LINEAR REGRESSION - MULTI-FEATURE MODEL
1. FEATURE PREPARATION
------------------------------
Input features: ['nsmiles', 'passengers', 'quarter', 'Year', 'large_ms', 'lf_ms', 'carrier_lg_idx']
Total features: 7

2. CREATE MODELS
------------------------------

3. HYPERPARAMETER TUNING
------------------------------
Parameter grid size: 24

4. CREATE PIPELINES
------------------------------

5. CROSS-VALIDATION SETUP
------------------------------
Cross-validator created with 3-fold CV

6. TRAINING MULTI-FEATURE MODEL
------------------------------
Training data: 195,235 rows
Test data: 49,108 rows
Training multi-feature model...

7. MODEL EVALUATION
------------------------------
Multi-feature Fare Prediction Model:
  RMSE: 64.8726
  R²: 0.3298
  MAE: 47.0739

8. BEST PARAMETERS
------------------------------
Best parameters:
  Regularization: 0.01
  Elastic Net: 0.5
  Max Iterations: 100

9. FEATURE IMPORTANCE
------------------------------
Features ranked by absolute coefficient val

In [7]:
# MODEL COMPARISON: Single vs Multi-Feature
print("=" * 60)
print("MODEL COMPARISON: SINGLE vs MULTI-FEATURE")
print("=" * 60)

# 1. COMPARISON TABLE
print("1. PERFORMANCE COMPARISON")
print("-" * 30)

print("FARE PREDICTION MODELS:")
print(f"{'Model':25} {'R²':>8} {'RMSE':>10} {'MAE':>10} {'Features':>10}")
print("-" * 65)

# Single-feature model (from previous cells)
print(f"{'Single-feature (distance)':25} {r2_fare:8.4f} {rmse_fare:10.4f} {mae_fare:10.4f} {'1':>10}")
print(f"{'Multi-feature (all vars)':25} {r2_multi:8.4f} {rmse_multi:10.4f} {mae_multi:10.4f} {'7':>10}")

# Calculate improvements
r2_improvement = ((r2_multi - r2_fare) / r2_fare) * 100
rmse_improvement = ((rmse_fare - rmse_multi) / rmse_fare) * 100
mae_improvement = ((mae_fare - mae_multi) / mae_fare) * 100

print("-" * 65)
print(f"{'IMPROVEMENT':25} {r2_improvement:8.1f}% {rmse_improvement:10.1f}% {mae_improvement:10.1f}% {'+6':>10}")

# 2. DETAILED ANALYSIS
print("\n2. DETAILED ANALYSIS")
print("-" * 30)

print("R² IMPROVEMENT:")
print(f"  Single-feature R²: {r2_fare:.4f} ({r2_fare*100:.1f}%)")
print(f"  Multi-feature R²: {r2_multi:.4f} ({r2_multi*100:.1f}%)")
print(f"  Improvement: {r2_improvement:.1f}% more variance explained")

print("\nRMSE IMPROVEMENT:")
print(f"  Single-feature RMSE: {rmse_fare:.4f}")
print(f"  Multi-feature RMSE: {rmse_multi:.4f}")
print(f"  Improvement: {rmse_improvement:.1f}% reduction in error")

print("\nMAE IMPROVEMENT:")
print(f"  Single-feature MAE: {mae_fare:.4f}")
print(f"  Multi-feature MAE: {mae_multi:.4f}")
print(f"  Improvement: {mae_improvement:.1f}% reduction in error")

# 3. FEATURE IMPORTANCE INSIGHTS
print("\n3. FEATURE IMPORTANCE INSIGHTS")
print("-" * 30)

print("Top 3 most important features:")
for i, (feature, coef) in enumerate(feature_importance[:3]):
    print(f"  {i+1}. {feature:15}: {coef:8.4f}")

print("\nFeature interpretation:")
for feature, coef in feature_importance:
    if feature == 'nsmiles':
        print(f"  Distance (nsmiles): {coef:8.4f} - {coef:.2f} dollars per mile")
    elif feature == 'passengers':
        print(f"  Passengers: {coef:8.4f} - {coef:.4f} dollars per passenger")
    elif feature == 'quarter':
        print(f"  Quarter: {coef:8.4f} - seasonal effect")
    elif feature == 'Year':
        print(f"  Year: {coef:8.4f} - temporal trend")
    elif feature == 'large_ms':
        print(f"  Large carrier market share: {coef:8.4f}")
    elif feature == 'lf_ms':
        print(f"  Low-cost carrier market share: {coef:8.4f}")
    elif feature == 'carrier_lg_idx':
        print(f"  Carrier type: {coef:8.4f} - carrier-specific effect")

# 4. BUSINESS IMPLICATIONS
print("\n4. BUSINESS IMPLICATIONS")
print("-" * 30)

print("SINGLE-FEATURE MODEL:")
print(f"  • Simple distance-based pricing")
print(f"  • Explains {r2_fare*100:.1f}% of fare variance")
print(f"  • Average error: ±${rmse_fare:.0f}")
print(f"  • Good for basic distance pricing")

print("\nMULTI-FEATURE MODEL:")
print(f"  • Comprehensive pricing model")
print(f"  • Explains {r2_multi*100:.1f}% of fare variance")
print(f"  • Average error: ±${rmse_multi:.0f}")
print(f"  • Better for complex pricing strategies")

# 5. RECOMMENDATIONS
print("\n5. RECOMMENDATIONS")
print("-" * 30)

if r2_improvement > 20:
    print("✅ STRONG RECOMMENDATION: Use multi-feature model")
    print("  • Significant improvement in accuracy")
    print("  • Worth the additional complexity")
elif r2_improvement > 10:
    print("✅ MODERATE RECOMMENDATION: Consider multi-feature model")
    print("  • Noticeable improvement in accuracy")
    print("  • Balance complexity vs. accuracy")
else:
    print("⚠️  WEAK RECOMMENDATION: Single-feature model may suffice")
    print("  • Small improvement in accuracy")
    print("  • Consider if complexity is worth it")

print(f"\n✅ Model comparison completed!")
print(f"📊 Multi-feature model shows {r2_improvement:.1f}% improvement in R²")
print(f"🎯 Use multi-feature model for production pricing systems")


MODEL COMPARISON: SINGLE vs MULTI-FEATURE
1. PERFORMANCE COMPARISON
------------------------------
FARE PREDICTION MODELS:
Model                           R²       RMSE        MAE   Features
-----------------------------------------------------------------


NameError: name 'r2_fare' is not defined

In [ ]:
# FINAL SUMMARY - Multi-Feature Analysis
print("=" * 60)
print("FINAL SUMMARY - MULTI-FEATURE ANALYSIS")
print("=" * 60)

# 1. ANALYSIS OVERVIEW
print("1. ANALYSIS OVERVIEW")
print("-" * 30)
print(f"Dataset: {df.count():,} observations")
print(f"Time period: 1993-2024")
print(f"Features analyzed: {len(input_features)} variables")
print(f"Target variable: Fare prediction")

# 2. FEATURE SUMMARY
print("\n2. FEATURE SUMMARY")
print("-" * 30)
print("Input features used:")
for i, feature in enumerate(input_features, 1):
    print(f"  {i}. {feature}")

print(f"\nFeature types:")
print(f"  • Numeric features: {len([f for f in input_features if f not in ['carrier_lg_idx']])}")
print(f"  • Categorical features: 1 (carrier_lg)")

# 3. MODEL PERFORMANCE SUMMARY
print("\n3. MODEL PERFORMANCE SUMMARY")
print("-" * 30)

print("SINGLE-FEATURE MODEL (Distance only):")
print(f"  • Features: 1 (nsmiles)")
print(f"  • R²: {r2_fare:.4f} ({r2_fare*100:.1f}%)")
print(f"  • RMSE: {rmse_fare:.4f}")
print(f"  • MAE: {mae_fare:.4f}")

print("\nMULTI-FEATURE MODEL (All variables):")
print(f"  • Features: {len(input_features)}")
print(f"  • R²: {r2_multi:.4f} ({r2_multi*100:.1f}%)")
print(f"  • RMSE: {rmse_multi:.4f}")
print(f"  • MAE: {mae_multi:.4f}")

# 4. KEY FINDINGS
print("\n4. KEY FINDINGS")
print("-" * 30)

print("IMPROVEMENT ACHIEVED:")
print(f"  • R² improvement: {r2_improvement:.1f}%")
print(f"  • RMSE reduction: {rmse_improvement:.1f}%")
print(f"  • MAE reduction: {mae_improvement:.1f}%")

print("\nMOST IMPORTANT FEATURES:")
for i, (feature, coef) in enumerate(feature_importance[:3], 1):
    print(f"  {i}. {feature}: coefficient = {coef:.4f}")

# 5. BUSINESS INSIGHTS
print("\n5. BUSINESS INSIGHTS")
print("-" * 30)

print("PRICING INSIGHTS:")
print(f"  • Distance remains important: {feature_importance[0][1]:.2f} dollars per mile")
print(f"  • Passenger volume impact: {feature_importance[1][1]:.4f} dollars per passenger")
print(f"  • Seasonal effects: Quarter coefficient = {feature_importance[2][1]:.4f}")

print("\nCOMPETITIVE INSIGHTS:")
for feature, coef in feature_importance:
    if 'ms' in feature:
        print(f"  • {feature}: {coef:.4f} (market share effect)")

# 6. RECOMMENDATIONS
print("\n6. RECOMMENDATIONS")
print("-" * 30)

print("FOR AIRLINE PRICING STRATEGY:")
print(f"  ✅ Use multi-feature model for production pricing")
print(f"  ✅ Distance-based pricing with {len(input_features)} variables")
print(f"  ✅ Consider seasonal and competitive factors")

print("\nFOR MODEL DEPLOYMENT:")
if r2_improvement > 15:
    print(f"  ✅ STRONG CASE: {r2_improvement:.1f}% improvement justifies complexity")
elif r2_improvement > 5:
    print(f"  ⚠️  MODERATE CASE: {r2_improvement:.1f}% improvement - evaluate trade-offs")
else:
    print(f"  ❌ WEAK CASE: {r2_improvement:.1f}% improvement may not justify complexity")

# 7. NEXT STEPS
print("\n7. NEXT STEPS")
print("-" * 30)
print("  • Deploy multi-feature model for fare prediction")
print("  • Monitor model performance in production")
print("  • Consider additional features (competition, seasonality)")
print("  • Implement ensemble methods for better accuracy")
print("  • Add real-time model retraining")

print(f"\n🎯 CONCLUSION:")
print(f"Multi-feature model with {len(input_features)} variables shows {r2_improvement:.1f}% improvement")
print(f"over single-feature model, making it suitable for production pricing systems.")
print(f"\n✅ Multi-feature analysis completed successfully!")


In [ ]:
# RANDOM FOREST MODEL - Distance, Quarter, Year Features
print("=" * 60)
print("RANDOM FOREST MODEL - DISTANCE, QUARTER, YEAR")
print("=" * 60)

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

# 1. FEATURE PREPARATION
print("1. FEATURE PREPARATION")
print("-" * 30)

# Chọn features: khoảng cách, quý, năm
rf_features = ['nsmiles', 'quarter', 'Year']
print(f"Selected features: {rf_features}")

# Assemble features
rf_assembler = VectorAssembler(inputCols=rf_features, outputCol="rf_features")

# Scale features
rf_scaler = StandardScaler(inputCol="rf_features", outputCol="rf_scaled_features", withStd=True, withMean=True)

# 2. CREATE RANDOM FOREST MODEL
print("\n2. CREATE RANDOM FOREST MODEL")
print("-" * 30)

rf = RandomForestRegressor(
    featuresCol="rf_scaled_features",
    labelCol="fare",
    predictionCol="fare_pred_rf",
    numTrees=100,
    maxDepth=10,
    maxBins=32,
    seed=42
)

print("Random Forest parameters:")
print(f"  Number of Trees: {rf.getNumTrees()}")
print(f"  Max Depth: {rf.getMaxDepth()}")
print(f"  Max Bins: {rf.getMaxBins()}")

# 3. HYPERPARAMETER TUNING
print("\n3. HYPERPARAMETER TUNING")
print("-" * 30)

# Create parameter grid for Random Forest
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100, 200]) \
    .addGrid(rf.maxDepth, [5, 10, 15, 20]) \
    .addGrid(rf.maxBins, [16, 32, 64]) \
    .addGrid(rf.minInstancesPerNode, [1, 5, 10]) \
    .build()

print(f"Parameter grid size: {len(rf_param_grid)}")

# 4. CREATE PIPELINE
print("\n4. CREATE PIPELINE")
print("-" * 30)

rf_pipeline = Pipeline(stages=[rf_assembler, rf_scaler, rf])

# 5. CROSS-VALIDATION
print("\n5. CROSS-VALIDATION SETUP")
print("-" * 30)

rf_evaluator = RegressionEvaluator(
    labelCol="fare",
    predictionCol="fare_pred_rf",
    metricName="rmse"
)

rf_cv = CrossValidator(
    estimator=rf_pipeline,
    estimatorParamMaps=rf_param_grid,
    evaluator=rf_evaluator,
    numFolds=3,
    seed=42,
    parallelism=2
)

print("Cross-validator created with 3-fold CV")

# 6. TRAIN MODEL
print("\n6. TRAINING RANDOM FOREST MODEL")
print("-" * 30)

# Split data
train_rf, test_rf = df.randomSplit([0.8, 0.2], seed=42)

print(f"Training data: {train_rf.count():,} rows")
print(f"Test data: {test_rf.count():,} rows")

print("Training Random Forest model...")
rf_cv_model = rf_cv.fit(train_rf)

# 7. EVALUATE MODEL
print("\n7. MODEL EVALUATION")
print("-" * 30)

# Make predictions
rf_predictions = rf_cv_model.transform(test_rf)

# Evaluate
rf_rmse = rf_evaluator.evaluate(rf_predictions)
rf_r2 = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_rf", metricName="r2").evaluate(rf_predictions)
rf_mae = RegressionEvaluator(labelCol="fare", predictionCol="fare_pred_rf", metricName="mae").evaluate(rf_predictions)

print("Random Forest Fare Prediction Model:")
print(f"  RMSE: {rf_rmse:.4f}")
print(f"  R²: {rf_r2:.4f}")
print(f"  MAE: {rf_mae:.4f}")

# 8. BEST PARAMETERS
print("\n8. BEST PARAMETERS")
print("-" * 30)

best_rf_model = rf_cv_model.bestModel
rf_stage = best_rf_model.stages[-1]

print("Best Random Forest parameters:")
print(f"  Number of Trees: {rf_stage.numTrees}")
print(f"  Max Depth: {rf_stage.maxDepth}")
print(f"  Max Bins: {rf_stage.maxBins}")
print(f"  Min Instances Per Node: {rf_stage.minInstancesPerNode}")

# 9. FEATURE IMPORTANCE
print("\n9. FEATURE IMPORTANCE")
print("-" * 30)

feature_importance = rf_stage.featureImportances.toArray()
rf_feature_importance = list(zip(rf_features, feature_importance))
rf_feature_importance.sort(key=lambda x: x[1], reverse=True)

print("Features ranked by importance:")
for feature, importance in rf_feature_importance:
    print(f"  {feature:15}: {importance:8.4f} ({importance*100:.2f}%)")

# 10. MODEL COMPARISON
print("\n10. MODEL COMPARISON")
print("-" * 30)

print("Linear Regression vs Random Forest:")
print(f"{'Model':25} {'R²':>8} {'RMSE':>10} {'MAE':>10}")
print("-" * 55)

# So sánh với Linear Regression (single feature)
print(f"{'Linear (distance only)':25} {r2_fare:8.4f} {rmse_fare:10.4f} {mae_fare:10.4f}")
print(f"{'Random Forest (3 features)':25} {rf_r2:8.4f} {rf_rmse:10.4f} {rf_mae:10.4f}")

# Calculate improvements
r2_improvement = ((rf_r2 - r2_fare) / r2_fare) * 100
rmse_improvement = ((rmse_fare - rf_rmse) / rmse_fare) * 100
mae_improvement = ((mae_fare - rf_mae) / mae_fare) * 100

print("-" * 55)
print(f"{'IMPROVEMENT':25} {r2_improvement:8.1f}% {rmse_improvement:10.1f}% {mae_improvement:10.1f}%")

# 11. BUSINESS INSIGHTS
print("\n11. BUSINESS INSIGHTS")
print("-" * 30)

print("RANDOM FOREST INSIGHTS:")
print(f"  • R²: {rf_r2:.4f} ({rf_r2*100:.1f}% of fare variance explained)")
print(f"  • Most important feature: {rf_feature_importance[0][0]} ({rf_feature_importance[0][1]*100:.1f}%)")
print(f"  • Second most important: {rf_feature_importance[1][0]} ({rf_feature_importance[1][1]*100:.1f}%)")
print(f"  • Third most important: {rf_feature_importance[2][0]} ({rf_feature_importance[2][1]*100:.1f}%)")

print(f"\n✅ Random Forest model completed!")
print(f"🎯 Features: {rf_features}")
print(f"📊 Performance: R² = {rf_r2:.4f}, RMSE = {rf_rmse:.4f}")
print(f"🔍 Most important: {rf_feature_importance[0][0]} ({rf_feature_importance[0][1]*100:.1f}%)")


RANDOM FOREST MODEL - DISTANCE, QUARTER, YEAR
1. FEATURE PREPARATION
------------------------------
Selected features: ['nsmiles', 'quarter', 'Year']

2. CREATE RANDOM FOREST MODEL
------------------------------
Random Forest parameters:
  Number of Trees: 100
  Max Depth: 10
  Max Bins: 32

3. HYPERPARAMETER TUNING
------------------------------
Parameter grid size: 108

4. CREATE PIPELINE
------------------------------

5. CROSS-VALIDATION SETUP
------------------------------
Cross-validator created with 3-fold CV

6. TRAINING RANDOM FOREST MODEL
------------------------------
Training data: 195,235 rows
Test data: 49,108 rows
Training Random Forest model...

7. MODEL EVALUATION
------------------------------
Random Forest Fare Prediction Model:
  RMSE: 63.5017
  R²: 0.3578
  MAE: 46.1637

8. BEST PARAMETERS
------------------------------
Best Random Forest parameters:


TypeError: 'int' object is not callable